In [1]:
from ase.build import molecule
from ase import Atoms, Atom
from ase.visualize import view
from numpy.linalg import norm
from ase.calculators.nwchem import NWChem
from ase.optimize import BFGS
import os
import numpy as np
from ase.calculators.emt import EMT
import matplotlib.pyplot as plt
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo
from ase.build import fcc111
from ase.constraints import FixAtoms
from ase.calculators.emt import EMT
import ase
import math
from numpy import linalg
import random
from ase.constraints import FixedLine
from ase.neb import NEB
from ase.db import connect
from ase.calculators.nwchem import NWChem
from ase.optimize import BFGS


class IUPAC:
    def alcohol(self, carbind, hind):
        self.atoms.symbols[hind] = "O"
        pos1x, pos1y, pos1z = self.atoms[carbind].position
        pos2x, pos2y, pos2z = self.atoms[hind].position
        self.atoms[hind].position = 1.43*pos2x-1.43*pos1x +pos1x, 1.43*pos2y-1.43*pos1y+pos1y, 1.43*pos2z-1.43*pos1z+pos1z
        pos2x, pos2y, pos2z = self.atoms[hind].position

        self.atoms.append(Atom("H", (-pos1x + pos2x*2, -pos1y + 2*pos2y, -pos1z + 2*pos2z)))
    def bromo(self, carbind, hind):
        pos1x, pos1y, pos1z = self.atoms[carbind].position
        pos2x, pos2y, pos2z = self.atoms[hind].position

        self.atoms.symbols[hind] = "Br"
        self.atoms[hind].position = 1.93*pos2x-1.93*pos1x+pos1x, 1.93*pos2y-1.93*pos1y+pos1y, 1.93*pos2z-1.93*pos1z+pos1z
    def chloro(self, carbind, hind):
        pos1x, pos1y, pos1z = self.atoms[carbind].position
        pos2x, pos2y, pos2z = self.atoms[hind].position

        self.atoms.symbols[hind] = "Cl"
        self.atoms[hind].position = 1.74*pos2x-1.74*pos1x+pos1x, 1.74*pos2y-1.74*pos1y+pos1y, 1.74*pos2z-1.74*pos1z+pos1z

    def fluoro(self, carbind, hind):
        pos1x, pos1y, pos1z = self.atoms[carbind].position
        pos2x, pos2y, pos2z = self.atoms[hind].position
        self.atoms.symbols[hind] = "F"

        self.atoms[hind].position = 1.35*pos2x-1.35*pos1x+pos1x, 1.35*pos2y-1.35*pos1y+pos1y, 1.35*pos2z-1.35*pos1z+pos1z
    def alkene(self, hind1, hind2):
        self.atoms.pop(hind2)
        self.atoms.pop(hind1)
    def alkyne(self, hind1, hind2, hind3, hind4):
        self.atoms.pop(hind4)
        self.atoms.pop(hind3)
        self.atoms.pop(hind2)
        self.atoms.pop(hind1)
    def aldehyde(self, hind1,hind2,hind3):
        self.atoms.symbols[hind2] ="O"
        self.atoms.pop(hind3)
    def carboxylic(self, cind, hind1,hind2,hind3):
        self.atoms.symbols[hind3] = "O"
        self.alcohol(cind, hind2)
        self.atoms.pop(hind1)
    def ketone(self, hind1, hind2):
        self.atoms.symbols[hind1] = "O"
        self.atoms.pop(hind2)
    def __init__(self, desiredrange):
        self.size = desiredrange
        self.ticker =1
        if desiredrange == 1:
            atoms = molecule("CH4")
            self.atoms = atoms
        if desiredrange == 2:
            atoms = molecule("C2H6")
            self.atoms = atoms
        if desiredrange>2:
            atoms = molecule("C2H6")
            atoms.symbols[7] = "C"
            norm1 = np.linalg.norm(atoms[1].position- atoms[0].position)
            atoms[7].position = norm1*(atoms[7].position- atoms[1].position)/np.linalg.norm(atoms[7].position-atoms[1].position)
            distX, distY, distZ = atoms[1].position - atoms[0].position
            posX, posY, posZ = atoms[7].position
            
            norm1 = np.linalg.norm(atoms[1].position - atoms[0].position)
            norm2 = np.linalg.norm(atoms[3].position - atoms[0].position)


            firsthydrogens1X, firsthydrogens1Y, firsthydrogens1Z = atoms[2].position - atoms[0].position
            firsthydrogens2X, firsthydrogens2Y, firsthydrogens2Z = atoms[4].position -atoms[0].position



            atoms.append(Atom("H", (posX +firsthydrogens1X, posY +firsthydrogens1Y, posZ +firsthydrogens1Z)))
            atoms.append(Atom("H", (posX +firsthydrogens2X, posY +firsthydrogens2Y, posZ +firsthydrogens2Z)))



            secondhydrogens1X, secondhydrogens1Y, secondhydrogens1Z = atoms[5].position - atoms[1].position
            secondhydrogens2X, secondhydrogens2Y, secondhydrogens2Z = atoms[6].position - atoms[1].position
            for i in range(desiredrange-3):
                atoms.append(Atom("C", (posX + distX, posY + distY, posZ + distZ)))
                posX, posY, posZ = atoms[10+3*i].position
                if i != 0:
                    distX, distY, distZ = atoms[7+3*(i)].position - atoms[4+3*(i)].position
                    norm1 = np.linalg.norm(atoms[7+3*(i)].position - atoms[4+3*(i)].position)
                else:
                    distX, distY, distZ = atoms[7].position - atoms[1].position
                    norm1 = np.linalg.norm(atoms[7].position - atoms[1].position)


                if i%2 == 1:
                    atoms.append(Atom("H", (posX +firsthydrogens1X, posY +firsthydrogens1Y, posZ +firsthydrogens1Z)))
                    atoms.append(Atom("H", (posX +firsthydrogens2X, posY +firsthydrogens2Y, posZ +firsthydrogens2Z)))
                    #gonna have to work with 3i
                else:
                    atoms.append(Atom("H", (posX +secondhydrogens1X, posY +secondhydrogens1Y, posZ +secondhydrogens1Z)))
                    atoms.append(Atom("H", (posX +secondhydrogens2X, posY +secondhydrogens2Y, posZ +secondhydrogens2Z)))
            atoms.append(Atom("H", (posX + distX*norm2/norm1, posY + distY*norm2/norm1, posZ + distZ*norm2/norm1)))
            self.atoms = atoms
    def __call__(self):
        return self.atoms
    def relax(self):
        labeler = 'newcalc' +'size' +str(self.size)+'no'+ str(self.ticker +8) +'/nwchem'
        calculator = NWChem(memory = '15 gb',label=labeler,dft={'xc': 'B3LYP','maxiter': 200,'convergence': {'damp': 50}, },basis='6-31+G*')
        self.atoms.set_calculator(calculator)
        opt = BFGS(self.atoms)
        opt.run(fmax=0.01)
        self.ticker +=1

        
        

        
        

def prefix(cs,group,mapper):
    pres = ['','', 'di', 'tri', 'tetra', 'penta', 'hexa', 'hepta', 'octa', 'nona' ,'deca', 'undeca', 'dodeca', 'trideca', 'tetradeca', 'pentadeca', 'hexadeca', 'heptadeca', 'octadeca','nonadeca', 'icosa']
    groups = [8,7,6,3,2,1,0]
    name = ''
    groupname = ['carboxy','oxo', 'oxo','hydroxy', '', '','fluoro','chloro','bromo']
    for i in range(len(groups)):
        if groups[i] in group:
            g = groups[i]
            indices = [i for i, x in enumerate(group) if x == g]
            if len(indices)==0:

                name += str(cs[mapper[group.index(groups[i])]])
            else:
                for I in range(len(indices)):
                    if I != 0:
                        name += ','
                    name += str(cs[mapper[indices[I]]])

            name += '-' + pres[len(indices)] +groupname[groups[i]] +'-'
    return name

def suffix(cs,group,mapper):
    groups = ["oic acid", "al", "one", "ol"]
    min_value = min(group)
    if min_value>3:
        return group, "e"
    min_index = group.index(min_value)
    group[min_index] = 10
    if cs[mapper[min_index]] ==1 and min_value!= 2:
        return group, groups[min_value]
    else:
        return group, '-'+str(cs[mapper[min_index]]) +"-" + groups[min_value]
def presuffix(cs, group,mapper):
    pres = ['','', 'di', 'tri', 'tetra', 'penta', 'hexa', 'hepta', 'octa', 'nona' ,'deca', 'undeca', 'dodeca', 'trideca', 'tetradeca', 'pentadeca', 'hexadeca', 'heptadeca', 'octadeca','nonadeca', 'icosa']
    if 4 in group:
        if len(cs)==2:
            return 'en'
        indices4 = [i for i, x in enumerate(group) if x == 4]
        indices5 = [i for i, x in enumerate(group) if x == 5]
        eneName = ''
        for i in range(len(indices4)):
            if i != 0:
                eneName += ','
            eneName += str(cs[mapper[indices4[i]]])

        eneName += '-' + pres[len(indices4)] +'en'
        yneName = ''
        for i in range(len(indices5)):
            if i != 0:
                yneName += ','
            yneName += str(cs[mapper[indices5[i]]])

        yneName += '-' + pres[len(indices5)] +'yn'

        if 5 in group:
            if indices4[0]< indices5[0]:
                return '-' + eneName +'e-'  + yneName
            else:
                return '-' + yneName+'e-' +eneName
        else:
            return '-' + eneName
    if 5 in group:
        if len(cs)==2:
            return 'yn'
        indices4 = [i for i, x in enumerate(group) if x == 4]
        indices5 = [i for i, x in enumerate(group) if x == 5]
        eneName = ''

        for i in range(len(indices4)):
            if i != 0:
                eneName += ','
            eneName += str(cs[ mapper[indices4[i]]])

        eneName += '-' + pres[len(indices4)] +'en'
        yneName = ''
        for i in range(len(indices5)):
            if i != 0:
                yneName += ','
            yneName += str( cs[mapper[ indices5[i] ]])

        yneName += '-' + pres[len(indices5)] +'yn'


        if 4 in group:
            if indices4[0]< indices5[0]:
                return '-' + eneName+'e-'  + yneName
            else:
                return '-'+ yneName+'e-' +eneName
        else:
            return '-' + yneName
    else:
        return "an"
def Greek(cs, group):
    greeks = ["","meth", "eth", "prop", "but", "pent", "hex", "hept", "oct", "non", "dec", "undec", "dodec", "tridec", "tetradec", "pentadec", "hexadec", "heptadec", "octadec", "nonadec", "eicos"]
    out = greeks[len(cs)]
    indices4, indices5 = [],[]
    if 4 in group:
        indices4 = [i for i, x in enumerate(group) if x == 4]
    if 5 in group:
        indices5 = [i for i, x in enumerate(group) if x == 5]
    if len(indices4)>1 or len(indices5)>1:
        out = out + "a"
    return out

def namer(size, group):
    groups = group.copy()
    cs = np.arange(1,size+1, 1).tolist()
    mapp = hydrogenlist(len(cs))
    mapp[0] = 0
    mapp[1] = 0
    mapp[2] = 0
    if size ==1:
        mapp[3] = 0
    else:
        for i in range(3,len(mapp)-1):
            mapp[i]= math.floor((i+1)/3)
        mapp[len(mapp)-1] = size-1


    for i in range(len(groups)):
        if groups[i]==4:
            groups[i] =12
        if groups[i]==5:
            groups[i]=13
        if groups[i] ==7:
            groups[i]=5
        if groups[i] ==8:
            groups[i] = 4

    minval = min(groups)
    min_index = groups.index(minval)
    for i in range(len(groups)):
        if groups[i] ==5:
            groups[i]=7
        if groups[i] ==4:
            groups[i] = 8
        if groups[i]==12:
            groups[i] =4
        if groups[i]==13:
            groups[i]=5

    if min_index> math.ceil(len(cs)/2)*2 and size !=1 :
        indeces4 = [i for i, x in enumerate(groups) if x == 4]
        indeces5 = [i for i, x in enumerate(groups) if x == 5]
        while indeces4:
            ind = indeces4[0]
            if ind ==0:
                if groups[3] ==10:
                    groups[3] =4
                    groups[0] =10
                else:
                    groups[4] = 4
                    groups[0] = 10
            elif ind ==len(groups)-1:
                donothing=0
            else:
                if groups[ind +2]==10:
                    groups[ind +2]=4
                    groups[ind]=10
                elif groups[ind +3]==10:
                    groups[ind +3]=4
                    groups[ind]=10
                elif groups[ind +4]==10 and ind+4==len(groups)-1:
                    groups[-1]=4
                    groups[ind]=10
            indeces4.pop(0)
        while indeces5:
            ind = indeces5[0]
            if ind ==0:
                if groups[3] ==10:
                    groups[3] =4
                    groups[0] =10
                else:
                    groups[4] = 4
                    groups[0] = 10
            elif ind ==len(groups)-1:
                donothing=0
            else:
                if groups[ind +2]==10:
                    groups[ind +2]=4
                    groups[ind]=10
                elif groups[ind +3]==10:
                    groups[ind +3]=4
                    groups[ind]=10
                elif groups[ind +4]==10 and ind+4==len(groups)-1:
                    groups[-1]=4
                    groups[ind]=10
            indeces5.pop(0)


        groups.reverse() #need to change the positions of bonds
        #at bond = 0 index will be at zero, move it to either 3 or 4 depending on which is ten, otherwise
        #move it up two indeces or three, at second to last move it up two three or four, on last don't move it




    gre = Greek(cs,groups)
    pres = presuffix(cs,groups,mapp)
    groups,suff = suffix(cs,groups,mapp)
    pref = prefix(cs,groups,mapp)

    nom = pref+gre+pres+suff
    return nom

def carblist(size):
    lis = [0,1]
    for i in range(size-2 ):
        lis.append(7+i*3)
    return lis

def externals(size):
    lis = [0]
    if size>1:
        lis.append(size-1)
    return lis
def internals(size):
    lis = []
    for i in range(1, size-1):
        lis.append(i)
    return lis
def hydrogenlist(size):
    hlist = [2,3,4]
    if size ==1:
        hlist.append(1)
    if size >1:
        hlist.append(5)
        hlist.append(6)
        if size==2:
            hlist.append(7)
        if size>2:
            for i in range(size-2):
                hlist.append(8+3*i)
                hlist.append(9+3*i)
            hlist.append(10+3*i)
    return hlist



In [48]:
def runner(nombre, reps):
    with open(nombre+'names.txt', 'w') as f:
            samplelinetotakeupsspace=0
    with open(nombre+'sizes.txt', 'w') as f:
        samplelinetotakeupsspace=0
    with open(nombre+'elements.txt', 'w') as f:
        samplelinetotakeupsspace=0
    with open(nombre+'positions.txt', 'w') as f:
            samplelinetotakeupsspace=0
    for idk in range(reps):
        for size in range(1,10):

            atoms = IUPAC(size)
            hsize = 2+2*size
            hlist = hydrogenlist(size)
            originalhlist = hydrogenlist(size)
            unticked = hydrogenlist(size)
            groups = hydrogenlist(size)
            for i in range(len(groups)):
                groups[i] =10
            ins = internals(size)
            outs = externals(size)
            carbs = carblist(size)

            bonds = np.ones(size-1)*2

            name = namer(size,groups)
            a = atoms()[0].position
            with open(nombre+'names.txt', 'a') as f:
                    f.write(name+'\n')
            with open(nombre+'sizes.txt', 'a') as f:
                f.write(str(len(atoms())) +'\n')
            with open(nombre+'elements.txt', 'a') as f:
                for atom in atoms():
                    f.write(str(atom.symbol) +',')
                f.write('\n')
            with open(nombre+'positions.txt', 'a') as f:
                for atom in atoms():
                    pos = atom.position-a
                    f.write(str(pos)+',')
                f.write('\n')
            while (hlist):
                lis = [0,1,2,3,4,5,6]
                if (outs):
                    lis.append(8)
                    lis.append(7)
                if len(ins)==0 or size ==1:
                    lis.pop(lis.index(6))
                if 2 not in bonds:
                    lis.pop(lis.index(5))
                if 1 not in bonds:
                    lis.pop(lis.index(4))

                i = random.choice(lis)

                if i == 0:
                    
                    picked = random.randrange(len(hlist))
                    if groups[originalhlist.index(hlist[picked] )]==10:
                        groups[originalhlist.index(hlist[picked] )] = 3
                    else:
                        groups[originalhlist.index(hlist[picked] )+1] = 3


                    if groups[0]<2 and hlist[picked] ==4:
                        carbon =1
                    else:
                        originalhlist.reverse()
                        unticked.reverse()
                        carbon = int(np.floor((unticked[originalhlist.index(hlist[picked])]-2)/3))
                        originalhlist.reverse()
                        unticked.reverse()
                        
                    if carbon ==-1:
                        carbon =0
                    atoms.alcohol(carbs[carbon], hlist[picked])
                    if carbon ==0:
                        if carbon in outs:
                            outs.pop(0)
                    if carbon ==size-1:
                        if carbon in outs:
                            outs.pop(outs.index(carbon))
                    if carbon in ins:
                        ins.pop(ins.index(carbon))
                    hlist.pop(picked)
                    if size==1:
                        outs = []

                    if carbon ==0 and 0 not in outs and size!=1:
                        bonds[0]+=-1
                    if carbon ==size-1 and size-1 not in outs and size>2:
                        bonds[size-2] +=-1
                    if 0<carbon<size-1:
                        bonds[carbon-1]+=-1
                        bonds[carbon]+=-1
                    if size==2 and carbon ==1:
                        bonds[0] +=-1
                if i ==1:
                    picked = random.randrange(len(hlist))
                    if groups[originalhlist.index(hlist[picked] )]==10:
                        groups[originalhlist.index(hlist[picked] )] = 8
                    else:
                        groups[originalhlist.index(hlist[picked] )+1] = 8



                    if groups[0]<2 and hlist[picked] ==4:
                        carbon =1
                    else:
                        originalhlist.reverse()
                        unticked.reverse()
                        carbon = int(np.floor((unticked[originalhlist.index(hlist[picked])]-2)/3))
                        originalhlist.reverse()
                        unticked.reverse()


                    if carbon ==-1:
                        carbon =0
                    atoms.bromo(carbs[carbon],hlist[picked])
                    if carbon ==0:
                        if carbon in outs:
                            outs.pop(0)
                    if carbon ==size-1:
                        if carbon in outs:
                            outs.pop(outs.index(carbon))
                    if carbon in ins:
                        ins.pop(ins.index(carbon))
                    hlist.pop(picked)
                    if size==1:
                        outs = []

                    if carbon ==0 and 0 not in outs and size!=1:
                        bonds[0]+=-1
                    if carbon ==size-1 and size-1 not in outs and size>2:
                        bonds[size-2] +=-1
                    if 0<carbon<size-1:
                        bonds[carbon-1]+=-1
                        bonds[carbon]+=-1
                    if size==2 and carbon ==1:
                        bonds[0] +=-1

                if i ==2:
                    picked = random.randrange(len(hlist))
                    if groups[originalhlist.index(hlist[picked] )]==10:
                        groups[originalhlist.index(hlist[picked] )] = 7
                    else:
                        groups[originalhlist.index(hlist[picked] )+1] = 7


                    if groups[0]<2 and hlist[picked] ==4:
                        carbon =1
                    else:
                        originalhlist.reverse()
                        unticked.reverse()
                        carbon = int(np.floor((unticked[originalhlist.index(hlist[picked])]-2)/3))
                        originalhlist.reverse()
                        unticked.reverse()


                    if carbon ==-1:
                        carbon =0
                    atoms.chloro(carbs[carbon], hlist[picked])
                    if carbon ==0:
                        if carbon in outs:
                            outs.pop(0)
                    if carbon ==size-1:
                        if carbon in outs:
                            outs.pop(outs.index(carbon))

                    if carbon in ins:
                        ins.pop(ins.index(carbon))
                    hlist.pop(picked)
                    if size==1:
                        outs = []

                    if carbon ==0 and 0 not in outs and size!=1:
                        bonds[0]+=-1
                    if carbon ==size-1 and size-1 not in outs and size>2:
                        bonds[size-2] +=-1
                    if 0<carbon<size-1:
                        bonds[carbon-1]+=-1
                        bonds[carbon]+=-1
                    if size==2 and carbon ==1:
                        bonds[0] +=-1
                if i==3:
                    picked = random.randrange(len(hlist))
                    if groups[originalhlist.index(hlist[picked] )]==10:
                        groups[originalhlist.index(hlist[picked] )] = 6
                    else:
                        groups[originalhlist.index(hlist[picked] )+1] = 6
                    g = groups.copy()
                    h = hlist[picked]


                    if groups[0]<2 and hlist[picked] ==4:
                        carbon =1
                    else:
                        originalhlist.reverse()
                        unticked.reverse()
                        carbon = int(np.floor((unticked[originalhlist.index(hlist[picked])]-2)/3))
                        originalhlist.reverse()
                        unticked.reverse()
                    if carbon ==-1:
                        carbon =0
                    atoms.fluoro(carbs[carbon], hlist[picked])
                    if carbon ==0:
                        if carbon in outs:
                            outs.pop(0)
                    if carbon ==size-1:
                        if carbon in outs:
                            outs.pop(outs.index(carbon))
                    if carbon in ins:
                        ins.pop(ins.index(carbon))
                    hlist.pop(picked)
                    if size==1:
                        outs = []

                    if carbon ==0 and 0 not in outs and size!=1:
                        bonds[0]+=-1
                    if carbon ==size-1 and size-1 not in outs and size>2:
                        bonds[size-2] +=-1
                    if 0<carbon<size-1:
                        bonds[carbon-1]+=-1
                        bonds[carbon]+=-1
                    if size==2 and carbon ==1:
                        bonds[0] +=-1
                if i ==8:

                    picked = random.randrange(len(outs))
                    if outs[picked] ==0:
                        hs = [2,3,4]
                        for I in range(hlist.index(4) +1, len(hlist)):
                            hlist[I] +=-1 #when popping an atom indexing gets messed
                        for I in range(originalhlist.index(4)+1, len(originalhlist)):
                            originalhlist[I]+=-1
                        originalhlist[0] =0
                        originalhlist[1] = 1
                        originalhlist[2] = 2
                        groups[0] = 1
                        for I in range(2,len(carbs)):
                            carbs[I] +=-1
                        if size!=1:
                            bonds[0] = 0
                    else:
                        groups[-1]=1
                        hs = []
                        for I in range(3):
                            hs.append(hlist[-1-I])

                        bonds[size-2] =0


                    atoms.aldehyde(hs[0], hs[1], hs[2])
                    outs.pop(picked)
                    for I in range(len(hs)):
                        hlist.pop(hlist.index(hs[I]))
                    if size==1:
                        hlist = []


                if i ==7:
                    picked = random.randrange(len(outs))
                    if outs[picked] ==0:
                        hs = [2,3,4]
                        for I in range(hlist.index(4) +1, len(hlist)):
                            hlist[I] +=-1
                        for I in range(originalhlist.index(4)+1, len(originalhlist)):
                            originalhlist[I]+=-1
                        originalhlist[0] =0
                        originalhlist[1] = 1
                        originalhlist[2] = 2
                        groups[0] = 0
                        for I in range(2,len(carbs)):
                            carbs[I] +=-1
                        if size!=1:
                            bonds[0] = 0
                    else:
                        hs = []
                        groups[-1] = 0
                        for I in range(3):
                            hs.append(hlist[-1-I])
                        bonds[size-2] =0

                    atoms.carboxylic(carbs[outs[picked]], hs[0], hs[1], hs[2])
                    outs.pop(picked)
                    for I in range(len(hs)):
                        hlist.pop(hlist.index(hs[I]))
                    if size==1:
                        hlist = []
                if i ==6:
                    picked = random.randrange(len(ins))
                    carbon = carbs[ins[picked]]

                    if ins[picked] ==1:
                        adder = 0
                        if groups[0]<2 or groups[0] ==4:
                            adder = -1
                        hydrogens = [carbon+4+adder, carbon+5+adder]
                    else:
                        hydrogens = [carbon +1, carbon+2]
                    for I in range(hlist.index(hydrogens[1])+1, len(hlist)):
                        hlist[I] +=-1
                    for I in range(originalhlist.index(hydrogens[1])+1, len(originalhlist)):
                        originalhlist[I] += -1
                    for I in range(ins[picked] +1, len(carbs)):
                        carbs[I] += -1

                    atoms.ketone(hydrogens[0], hydrogens[1])
                    groups[originalhlist.index(hydrogens[0])] =2
                    for I in range(len(hydrogens)):
                        hlist.pop(hlist.index(hydrogens[I]))

                    if 0<ins[picked]<size-1:
                        bonds[ins[picked]-1]+=-2 #this didn't fire for some reason
                        bonds[ins[picked]]+=-2

                    ins.pop(picked)
                #need alkenese and alkynes
                if i ==5:
                    indices = [i for i, x in enumerate(bonds) if x == 2]

                    bond = indices[random.randrange(len(indices))]
                    carbons = [bond, bond+1] # i think
                    Hs=[]
                    if bond ==0:
                        if size!=2:
                            if 2 in hlist:
                                Hs.append(2)
                            if 3 in hlist:
                                Hs.append(3)
                            if 4 in hlist and len(Hs) ==1:
                                Hs.append(4)
                            Hs.append(5)
                            Hs.append(6)
                        else:
                            if 2 in hlist:
                                Hs.append(2)
                            if 3 in hlist:
                                Hs.append(3)
                            if 4 in hlist and len(Hs) ==1:
                                Hs.append(4)
                            if 5 in hlist:
                                Hs.append(5)
                            if 6 in hlist:
                                Hs.append(6)
                            if 7 in hlist and len(Hs)==3:
                                Hs.append(7)
                        if 0 in outs:
                            outs.pop(outs.index(0))
                        if size==2:
                            if 1 in outs:
                                outs.pop(outs.index(1))
                        else:
                            ins.pop(ins.index(1))
                        groups[0] = 5
                    elif bond ==1:
                        adder = 0
                        if groups[0]<2 or groups[0] ==4:
                            adder = -1
                        Hs.append(5+adder)
                        Hs.append(6+adder)
                        if size!=3:
                            Hs.append(carbs[2]+1)
                            Hs.append(carbs[2]+2) #add in case where the one to the right is the last one

                            ins.pop(ins.index(1))
                            ins.pop(ins.index(2))
                        else:
                            if carbs[carbons[1]]+1 in hlist:
                                Hs.append(carbs[carbons[1]]+1)
                            if carbs[carbons[1]]+2 in hlist:
                                Hs.append(carbs[carbons[1]]+2)
                            if carbs[carbons[1]]+3 in hlist and len(Hs) ==3:
                                Hs.append(carbs[carbons[1]]+3)
                            ins.pop(ins.index(1))
                            if size-1 in outs:
                                outs.pop(outs.index(size-1))
                        groups[3] = 5

                    elif bond ==size-2 and size>3:
                        Hs.append(carbs[carbons[0]]+1)
                        Hs.append(carbs[carbons[0]]+2)
                        if carbs[carbons[1]]+1 in hlist:
                            Hs.append(carbs[carbons[1]]+1)
                        if carbs[ carbons[1] ]+2 in hlist:
                            Hs.append(carbs[carbons[1]]+2)
                        if carbs[carbons[1] ]+3 in hlist and len(Hs) ==3:
                            Hs.append(carbs[carbons[1]]+3)

                        ins.pop(ins.index(size-2))
                        if size-1 in outs:
                            outs.pop(outs.index(size-1))
                        groups[len(groups)-5] = 5 #does this make sense? ()
                    else:
                        Hs = [carbs[carbons[0]]+1, carbs[carbons[0]] +2, carbs[carbons[1]]+1, carbs[carbons[1]]+2]
                        ins.pop(ins.index(bond))
                        ins.pop(ins.index(bond+1))
                        groups[2*(carbons[0]-1)+3] = 5 #this here doesn't make any sense
                        #need the groups function

                    for I in range(hlist.index(Hs[3])+1, len(hlist)):
                        hlist[I] +=-4 #need to not increment the chosen hs add to these on hlist and original hlist
                    for I in range(originalhlist.index(Hs[3])+1, len(originalhlist)):
                        originalhlist[I] += -4
                    if bond ==0:
                        hlist[hlist.index(4)]+=-2
                        originalhlist[originalhlist.index(4)]+=-2
                    for I in range(carbons[1] +1, len(carbs)):
                        carbs[I] += -4
                    if carbons[1]!=1:
                        carbs[carbons[1]]+=-2

                    atoms.alkyne(Hs[0],Hs[1],Hs[2],Hs[3])
                    for I in range(len(Hs)):
                        hlist.pop(hlist.index(Hs[I]))


                    bonds[bond] = 0
                    if bond>0:
                        bonds[bond-1] =0
                    if bond<size-2:
                        bonds[bond+1]=0
                if i ==4:
                    indices = [i for i, x in enumerate(bonds) if x >= 1]

                    bond = indices[random.randrange(len(indices))]
                    carbons = [bond, bond+1] # i think
                    Hs=[]

                    if bond ==0:
                        if 2 in hlist:
                            Hs.append(2)
                        if 3 in hlist and len(Hs) ==0:
                            Hs.append(3)
                        if 4 in hlist and len(Hs) ==0:
                            Hs.append(4)
                        if 5 in hlist:
                            Hs.append(5)
                        if 6 in hlist and len(Hs)==1:
                            Hs.append(6)
                        if size==2:
                            if 7 in hlist and len(Hs)==1:
                                Hs.append(7)

                        if 0 in outs:
                            outs.pop(outs.index(0))
                        if size==2:
                            if 1 in outs:
                                outs.pop(outs.index(1))
                        else:
                            if 1 in ins:
                                ins.pop(ins.index(1))
                        groups[0] = 4
                    elif bond ==1:
                        adder = 0
                        if groups[0]<2 or groups[0] ==4:
                            adder = -1
                        if 5+adder in hlist:
                            Hs.append(5+adder)
                        if 6+adder in hlist and len(Hs)==0:
                            Hs.append(6+adder)

                        if size!=3:
                            if carbs[2]+1 in hlist:
                                Hs.append(carbs[2]+1)
                            if carbs[2] +2 in hlist and len(Hs)==1:
                                Hs.append(carbs[2]+2) #add in case where the one to the right is the last one
                            if 1 in ins:
                                ins.pop(ins.index(1))
                            if 2 in ins:
                                ins.pop(ins.index(2))
                        else:
                            if carbs[carbons[1]]+1 in hlist:
                                Hs.append(carbs[carbons[1]]+1)
                            if carbs[carbons[1]]+2 in hlist and len(Hs)==1:
                                Hs.append(carbs[carbons[1]]+2)
                            if carbs[carbons[1]]+3 in hlist and len(Hs) ==1:
                                Hs.append(carbs[carbons[1]]+3)
                            if 1 in ins:
                                ins.pop(ins.index(1))
                            if size-1 in outs:
                                outs.pop(outs.index(size-1))
                        groups[3] = 4

                    elif bond ==size-2 and size>3:
                        if carbs[carbons[0]]+1 in hlist:
                            Hs.append(carbs[carbons[0]]+1)
                        if carbs[carbons[0]]+2 in hlist and len(Hs)==0:
                            Hs.append(carbs[carbons[0]]+2)
                        if carbs[carbons[1]]+1 in hlist:
                            Hs.append(carbs[carbons[1]]+1)
                        if carbs[ carbons[1] ]+2 in hlist and len(Hs)==1:
                            Hs.append(carbs[carbons[1]]+2)
                        if carbs[carbons[1] ]+3 in hlist and len(Hs) ==1:
                            Hs.append(carbs[carbons[1]]+3)

                        if size-2 in ins:
                            ins.pop(ins.index(size-2))
                        if size-1 in outs:
                            outs.pop(outs.index(size-1))
                        groups[len(groups)-5] = 4 #does this make sense? ()
                    else:
                        if carbs[carbons[0]]+1 in hlist:
                            Hs.append(carbs[carbons[0]]+1)
                        if carbs[carbons[0]]+2 in hlist and len(Hs) ==0:
                            Hs.append(carbs[carbons[0]]+2)
                        if carbs[carbons[1]]+1 in hlist:
                            Hs.append(carbs[carbons[1]]+1)
                        if carbs[ carbons[1] ]+2 in hlist and len(Hs)==1:
                            Hs.append(carbs[carbons[1]]+2)

                        if bond in ins:
                            ins.pop(ins.index(bond))
                        if bond+1 in ins:
                            ins.pop(ins.index(bond+1))
                        groups[2*(carbons[0]-1)+3] = 4 #this here doesn't make any sense
                        #need the groups function

                    if bond ==0:
                        if Hs[0] ==2:
                            if 3 in hlist:
                                hlist[hlist.index(3)] +=-1
                                originalhlist[originalhlist.index(3)] +=-1
                            if 4 in hlist:
                                hlist[hlist.index(4)] +=-1
                                originalhlist[originalhlist.index(4)] +=-1
                        if Hs[0] ==3:
                            if 4 in hlist:
                                hlist[hlist.index(4)] +=-1
                                originalhlist[originalhlist.index(4)] +=-1
                    elif bond ==1:
                        adder = 0
                        if groups[0]<2 or groups[0] ==4:
                            adder = -1
                        if Hs[0] == 5+adder:
                            if 6 + adder in hlist:
                                hlist[hlist.index(6+adder)]+=-1

                    else:
                        if Hs[0] == carbs[bond]+1:
                            if Hs[0]+1 in hlist:
                                hlist[hlist.index(Hs[0])+1]+=-1

                    for I in range(hlist.index(Hs[1])+1, len(hlist)):
                        hlist[I] +=-2 #need to not increment the chosen hs add to these on hlist and original hlist
                    for I in range(originalhlist.index(Hs[1])+1, len(originalhlist)):
                        originalhlist[I] += -2
                    #need to increment the hydrogen on the first carbon that wasn't picked if it is larger than the one that was
                    #we also might need to increment two hydrogens. Also need to increment the carbon if not bond 0



                    for I in range(carbons[1] +1, len(carbs)):
                        carbs[I] += -2
                    if carbons[1]!=1:
                        carbs[carbons[1]]+=-1

                    atoms.alkene(Hs[0],Hs[1])
                    for I in range(len(Hs)):
                        hlist.pop(hlist.index(Hs[I]))


                    bonds[bond] = 0
                    if bond>0:
                        bonds[bond-1] +=-1
                    if bond<size-2:
                        bonds[bond+1]+=-1

                name = namer(size,groups)
                with open(nombre+'names.txt', 'a') as f:
                    f.write(name+'\n')
                with open(nombre+'sizes.txt', 'a') as f:
                    f.write(str(len(atoms())) +'\n')
                with open(nombre+'elements.txt', 'a') as f:
                    for atom in atoms():
                        f.write(str(atom.symbol) +',')
                    f.write('\n')
                with open(nombre+'positions.txt', 'a') as f:
                    for atom in atoms():
                        pos = atom.position-a
                        f.write(str(pos)+',')
                    f.write('\n')

In [54]:
runner("train", 10000)

In [55]:
runner("test", 1000)

<Popen: returncode: None args: ['C:\\Users\\Manuel\\anaconda3\\python.exe', ...>